In [37]:
import requests
import json
import urllib.parse

In [25]:
s = requests.Session()

In [26]:
baseAddress = "https://learning.whchem.com:6443/"
defaultHeader = {"User-Agent": "Mozilla /5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1"}
user = {"username": "00024849", "password": "ytpu2021"}
s.headers.update(defaultHeader)


In [91]:
#登录，并保存登录信息
s.headers.update({"authorization":s.post(baseAddress+"Api/User/Login",data=user).json()["data"]['TokenID']})

#获取未完成的任务列表
mission_nopass = list(filter(lambda e: e["LimitIntegral"] != e["Integral"] and e["TaskType"] != 12, s.get(baseAddress+"Api/Common/Task/GetTaskList", data={"taskGroup": "0"}).json()["data"]["list"]))

#获取闯关答题未完成列表
breakThrough_nopass_id = [x["PointLevelId"] for x in filter(lambda e: e["MaxPointNumber"] != e["QuestionNumber"], s.post(baseAddress+"Api/PointAnswer/GetPointAnswerDetail", data={"pageSize": "2000", "pageIndex": "0"}).json()["data"]['list'])]

#获取闯关答题题目
breakThrough_subject = s.post(baseAddress+"Api/PointAnswer/GetPointAnswerQuestion", data={"level": breakThrough_nopass_id[0], "isExercise": "false"}).json()


In [92]:
tmlist = breakThrough_subject["data"]["list"]
print(tmlist)
tmdata = [({
    "tmid": x.pop("Tm_ID"), 
    "txstr": x.pop("Tm_BaseTx"), 
    "title": x.pop("Title"),
    "options": x.pop("Options"),
}
) for x in tmlist]
#tmid_list

[{'TmSourceType': 0, 'Tk_ID': 657, 'Tm_ID': 49045, 'Tm_Tx': 593, 'Tm_BaseTx': '单选类', 'Parent_ID': None, 'Relate_Tm': None, 'Title': '液环泵可达到的极限真空范围为（）。', 'Tm_Key': '液环泵基础知识', 'Options': '2KPa～4KPa;2MPa～4MPa;200Pa～400Pa;20Pa～40Pa', 'Answers': 'Z', 'Difficulty': '易', 'Remark': '', 'Files': '', 'FileUrl': '', 'FileAnswerTemplate': None, 'AllowStudentUploadAnswer': 0, 'UserAnswer': '', 'Mark': None, 'UserAnswerFile': None, 'AnswerTime': '', 'AllowReport': 0, 'AnswerRight': 0, 'UserScore': 0.0, 'Tm_Tx_Str': '单选题', 'Score': 1.0, 'AttachFileInfo': None, 'QaResult': None}, {'TmSourceType': 0, 'Tk_ID': 657, 'Tm_ID': 48949, 'Tm_Tx': 592, 'Tm_BaseTx': '判断类', 'Parent_ID': None, 'Relate_Tm': None, 'Title': '长时间关闭磁力泵出口阀，泵可以正常运行。', 'Tm_Key': '磁力泵', 'Options': '对;错', 'Answers': 'Z', 'Difficulty': '易', 'Remark': '', 'Files': '', 'FileUrl': '', 'FileAnswerTemplate': None, 'AllowStudentUploadAnswer': 0, 'UserAnswer': '', 'Mark': None, 'UserAnswerFile': None, 'AnswerTime': '', 'AllowReport': 0, 'AnswerRigh

level=6d1d71e2-001f-4985-9ed0-3a1018adc3d7&answer=%5B%7B%22tmid%22%3A48950%2C%22answer%22%3A%22B%22%7D%2C%7B%22tmid%22%3A49158%2C%22answer%22%3A%22C%22%7D%2C%7B%22tmid%22%3A49175%2C%22answer%22%3A%22A%22%7D%5D&second=21

In [93]:
answersubmit = "level=6d1d71e2-001f-4985-9ed0-3a1018adc3d7&answer=%5B%7B%22tmid%22%3A48950%2C%22answer%22%3A%22B%22%7D%2C%7B%22tmid%22%3A49158%2C%22answer%22%3A%22C%22%7D%2C%7B%22tmid%22%3A49175%2C%22answer%22%3A%22A%22%7D%5D&second=21"
urllib.parse.unquote(answersubmit)


'level=6d1d71e2-001f-4985-9ed0-3a1018adc3d7&answer=[{"tmid":48950,"answer":"B"},{"tmid":49158,"answer":"C"},{"tmid":49175,"answer":"A"}]&second=21'

In [94]:
#获取答案
answerlist = []
with open ("tmdict.json", "r", encoding="utf-8") as f:
    tm = json.load(f)
    for gettm in tmdata:
        answer = {}
        answer["tmid"] = gettm["tmid"]
        answer["answer"] = tm[gettm["title"]]["answers"].replace(";", ",")
        answerlist.append(answer)
print(str(answerlist))
submit = s.post(baseAddress+"Api/PointAnswer/SubmitPointAnswer", data={"level": breakThrough_nopass_id[0], "answer": str(answerlist), "second": "20"})
#获取闯关答题未完成列表
breakThrough_nopass_id = [x["PointLevelId"] for x in filter(lambda e: e["MaxPointNumber"] != e["QuestionNumber"], s.post(baseAddress+"Api/PointAnswer/GetPointAnswerDetail", data={"pageSize": "2000", "pageIndex": "0"}).json()["data"]['list'])]


[{'tmid': 49045, 'answer': 'A'}, {'tmid': 48949, 'answer': 'B'}, {'tmid': 48937, 'answer': 'A'}]


In [103]:
#获取闯关答题未完成列表
breakThrough_nopass = [x["PointLevelId"] for x in filter(lambda e: e["CanJoin"] == True, s.post(baseAddress+"Api/PointAnswer/GetPointAnswerDetail", data={"pageSize": "2000", "pageIndex": "0"}).json()["data"]["list"])]
breakThrough_nopass

['232fc123-c25d-49c0-afce-e4429827e270']